# --- Day 10: Pipe Maze ---

## --- Part One ---

You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.

You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "[Hot Springs](https://en.wikipedia.org/wiki/Hot_spring)" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.

The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.

Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of **tiles**:

- `|` is a **vertical pipe** connecting north and south.
- `-` is a **horizontal pipe** connecting east and west.
- `L` is a **90-degree bend** connecting north and east.
- `J` is a **90-degree bend** connecting north and west.
- `7` is a **90-degree bend** connecting south and west.
- `F` is a **90-degree bend** connecting south and east.
- `.` is **ground**; there is no pipe in this tile.
- `S` is the **starting position** of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.

Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is **one large, continuous loop**.

For example, here is a square loop of pipe:

```
.....
.F-7.
.|.|.
.L-J.
.....
```

If the animal had entered this loop in the northwest corner, the sketch would instead look like this:

```
.....
.S-7.
.|.|.
.L-J.
.....
```

In the above diagram, the `S` tile is still a 90-degree `F` bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that **aren't connected to the loop**! This sketch shows the same loop as above:

```
-L|F7
7S-7|
L|7||
-L-J|
L|-JF
```

In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to `S`, pipes those pipes connect to, pipes **those** pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including `S`, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:

```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```

Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:

```
7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
```

If you want to **get out ahead of the animal**, you should find the tile in the loop that is **farthest** from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps **along the loop** to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:

```
.....
.S-7.
.|.|.
.L-J.
.....
```

You can count the distance each tile in the loop is from the starting point like this:

```
.....
.012.
.1.3.
.234.
.....
```

In this example, the farthest point from the start is **`4`** steps away.

Here's the more complex loop again:

```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```

Here are the distances for each tile on that loop:

```
..45.
.236.
01.78
14567
23...
```

Find the single giant loop starting at `S`. **How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?**

In [12]:
import numpy as np

def get_start_location(rows):
    for r_i, row in enumerate(rows):
        for c_i, col in enumerate(row):
            if col == 'S':
                return [r_i, c_i]

def find_connecting_pipes(rows, start_coord):
    s_r = start_coord[0]
    s_c = start_coord[1]

    connecting_pipes = []
    
    # check left
    if (s_c != 0) and (rows[s_r][s_c - 1] in ['L', 'F', '-']):
        connecting_pipes.append([[s_r, s_c - 1], rows[s_r][s_c - 1]])
    
    # check right
    if (s_c != len(rows[0]) - 1) and (rows[s_r][s_c + 1] in ['J', '7', '-']):
        connecting_pipes.append([[s_r, s_c + 1], rows[s_r][s_c + 1]])

    # check up
    if (s_r != 0) and (rows[s_r - 1][s_c] in ['7', 'F', '|']):
        connecting_pipes.append([[s_r - 1, s_c], rows[s_r - 1][s_c]])

    # check down
    if (s_r != len(rows) - 1) and (rows[s_r + 1][s_c] in ['L', 'J', '|']):
        connecting_pipes.append([[s_r + 1, s_c], rows[s_r + 1][s_c]])
    
    return connecting_pipes

def get_directions(symbol):
    if symbol == 'J':
        return [[0, -1], [-1, 0]]
    if symbol == 'L':
        return [[0, 1], [-1, 0]]
    if symbol == '7':
        return [[0, -1], [1, 0]]
    if symbol == 'F':
        return [[0, 1], [1, 0]]
    if symbol == '|':
        return [[1, 0], [-1, 0]]
    if symbol == '-':
        return [[0, -1], [0, 1]]

# Load input
with open('day_10_input.txt', 'r') as input:
    lines = input.readlines()

rows = [line[:-1] for line in lines]

# Get first connection
start_coord = get_start_location(rows)
connection = find_connecting_pipes(rows, start_coord)

# Loop variables
last_location = np.array(start_coord)
current_location = np.array(connection[0][0])
step_count = 1

current_symbol = connection[0][1]

while current_symbol != 'S':
    last_direction = last_location - current_location
    possible_directions = get_directions(current_symbol)
    possible_directions = [i for i in possible_directions if (i != last_direction).any()]

    last_location = current_location
    current_location = current_location + possible_directions[0]
    current_symbol = rows[current_location[0]][current_location[1]]

    step_count += 1

print(step_count/2)

80.0


## --- Part Two ---

You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is **within the area enclosed by the loop**?

To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:

```
...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
```

The above loop encloses merely **four tiles** - the two pairs of `.` in the southwest and southeast (marked `I` below). The middle `.` tiles (marked `O` below) are **not** in the loop. Here is the same loop again with those regions marked:

```
...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....
```

In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, `I` is still within the loop and `O` is still outside the loop:

```
..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........
```

In both of the above examples, **`4`** tiles are enclosed by the loop.

Here's a larger example:

```
.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
```

The above sketch has many random bits of ground, some of which are in the loop (`I`) and some of which are outside it (`O`):

```
OF----7F7F7F7F-7OOOO
O|F--7||||||||FJOOOO
O||OFJ||||||||L7OOOO
FJL7L7LJLJ||LJIL-7OO
L--JOL7IIILJS7F-7L7O
OOOOF-JIIF7FJ|L7L7L7
OOOOL7IF7||L7|IL7L7|
OOOOO|FJLJ|FJ|F7|OLJ
OOOOFJL-7O||O||||OOO
OOOOL---JOLJOLJLJOOO
```

In this larger example, **`8`** tiles are enclosed by the loop.

Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:

```
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
```

Here are just the tiles that are **enclosed by the loop** marked with I:

```
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
```

In this last example, **`10`** tiles are enclosed by the loop.

Figure out whether you have time to search for the nest by calculating the area within the loop. **How many tiles are enclosed by the loop?**

In [59]:
import skimage
import numpy as np

def get_start_location(rows):
    for r_i, row in enumerate(rows):
        for c_i, col in enumerate(row):
            if col == 'S':
                return [r_i, c_i]

def find_connecting_pipes(rows, start_coord):
    s_r = start_coord[0]
    s_c = start_coord[1]

    connecting_pipes = []
    
    # check left
    if (s_c != 0) and (rows[s_r][s_c - 1] in ['L', 'F', '-']):
        connecting_pipes.append([[s_r, s_c - 1], rows[s_r][s_c - 1]])
    
    # check right
    if (s_c != len(rows[0]) - 1) and (rows[s_r][s_c + 1] in ['J', '7', '-']):
        connecting_pipes.append([[s_r, s_c + 1], rows[s_r][s_c + 1]])

    # check up
    if (s_r != 0) and (rows[s_r - 1][s_c] in ['7', 'F', '|']):
        connecting_pipes.append([[s_r - 1, s_c], rows[s_r - 1][s_c]])

    # check down
    if (s_r != len(rows) - 1) and (rows[s_r + 1][s_c] in ['L', 'J', '|']):
        connecting_pipes.append([[s_r + 1, s_c], rows[s_r + 1][s_c]])
    
    return connecting_pipes

def get_directions(symbol):
    if symbol == 'J':
        return [[0, -1], [-1, 0]]
    if symbol == 'L':
        return [[0, 1], [-1, 0]]
    if symbol == '7':
        return [[0, -1], [1, 0]]
    if symbol == 'F':
        return [[0, 1], [1, 0]]
    if symbol == '|':
        return [[1, 0], [-1, 0]]
    if symbol == '-':
        return [[0, -1], [0, 1]]

# Load input
with open('day_10_input.txt', 'r') as input:
    lines = input.readlines()

rows = [line[:-1] for line in lines]

# Get first connection
start_coord = get_start_location(rows)
connection = find_connecting_pipes(rows, start_coord)

# Loop variables
last_location = np.array(start_coord)
current_location = np.array(connection[0][0])
step_count = 1

current_symbol = connection[0][1]
expanded_grid = np.ones((len(rows) * 2 + 1, len(rows[0]) * 2 + 1))

expanded_grid[start_coord[0] * 2 + 1][start_coord[1] * 2 + 1] = 2
expanded_grid[current_location[0] * 2 + 1][current_location[1] * 2 + 1] = 2
expanded_grid[(start_coord[0]+current_location[0]) + 1][(start_coord[1]+current_location[1]) + 1] = 2

while current_symbol != 'S':
    last_direction = last_location - current_location
    possible_directions = get_directions(current_symbol)
    possible_directions = [i for i in possible_directions if (i != last_direction).any()]

    last_location = current_location
    current_location = current_location + possible_directions[0]
    current_symbol = rows[current_location[0]][current_location[1]]

    step_count += 1

    eg_r = 1 + (current_location[0] * 2)
    eg_c = 1 + (current_location[1] * 2)

    expanded_grid[eg_r][eg_c] = 2

    eg_r = 1 + (current_location[0] * 2) - possible_directions[0][0]
    eg_c = 1 + (current_location[1] * 2) - possible_directions[0][1]

    expanded_grid[eg_r][eg_c] = 2

elements = skimage.measure.label(expanded_grid, background=False, connectivity=1)
first_pixel = elements[0,0] 

connected_elements = skimage.measure.regionprops(elements)
outside_elements = connected_elements[first_pixel - 1]

for r_i, line in enumerate(outside_elements.image):
    for c_i, element in enumerate(line):
        if element:
          expanded_grid[r_i][c_i] = 2

normal_grid = []
for r_i, row in enumerate(expanded_grid):
    if r_i % 2 == 1:
        new_row = []
        for c_i, col in enumerate(row):
            if c_i % 2 == 1:
                new_row.append(col)
        normal_grid.append(new_row)

inside_elements = 0
for line in normal_grid:
    for element in line:
        if element == 1:
            inside_elements += 1

print(inside_elements)

265
